# Homework 1

First name:

Last name:


---

The two exercises follow each other.

You will probably need to use an optimization library (like `Optim.jl`) and a differentiation library (`Forward.jl` or `Zygote.jl`) as well as a plotting library (`Plots.jl` or `SimplePlots.jl`) but feel free to install other libraries you might find useful.

You must send me your work before May 16th. You can send it on zulip or to `pwinant@escp.eu` with subject starting by `[eco309]`.

Don't hesitate to ask questions or help each other. Copying/pasting code is forbidden though.

---

## Solow model

This model tries to explain long-term growth, its relation to productivity, population growth and the speed of convergence.

The setup is the following:

- a country accumulates capital $k_t$
- population $n_t$ grows at rate $g$ (so that $n_{t+1}=g n_t$)
- capital and labour are combined to produce $y_t = A F(k_t, n_t)$ where $A>0$ is total factor productivity.
- capital depreciates at rate $\delta \in ]0,1[$. Its law of motion is $k_{t+1}=(1-\delta)k_{t}+i_{t}$ where $i_t$ is the amount invested at time $t$
- production is either consumed or invested: $y_t = c_t + i_t$

We choose a Cobb-Douglas specification with constant returns to scale $F(k_t, n_t) = k_t^{\alpha} n_t^{1-\alpha}$.


Here we follow the Solow-Swan specification and make the simplifying assumption that there is a fixed fraction $s\in[0,1[$ of income, which is saved and invested in every period. In other words investment (or savings) is given by:

$$i_t = s y_t$$

As a result, the dynamic of capital and all other variables will be backward looking and can be simulated easily.

__Create a dictionary to hold parameter values.__

 (note: such a dictionary can be defined as `p = Dict(:a=>0.9, :b=>4)` and accessed with brackets `p[:a]` or `p[:b]`) (note2: symbols like `:a` or `:b` are lightweight string. The content must be a valid julia identifier so that it is valid to write `a=p[:a]`)

In [ ]:
# We'll create a dictionary of parameters
p = Dict(
    :α=>0.3,
    :β=>0.96,
    :δ=>0.1,
    :g=>1.01,
    :s=>0.2
)

__Detrend the equations of the model w.r.t. population growth. Denote the detrended variables with a hat (in the code we assume all variables are detrended and ignore the hat).__

Our goal is to compute the steady-state and assess its stability.

__Compute a function `f` which returns the capital $\hat{k}_{t+1}$ as a function of $\hat{k_t}$. It is suggested to define `f(k::Float64, p::Dict{Symbol, Float64})` where `p` is a dictionary of parameters.__

__Starting from an initial level $\hat{k}_0$ compute successive iterates of `f` to find the long-run level $\overline{k}$ of capital per capita. (Bonus: produce a nice plot of the convergence)__

simulate (generic function with 4 methods)

__Propose and implement a way to accelerate convergence__

__What factors affect the steady-state level of capital?__

__Study the stability of $f$ around $\overline{k}$ (hint: compute the right derivative). Which factors affect the speed of convergence towards the steady-state?__

__Compute the steady-state directly using a Newton method and compare convergence speed.__

__Suppose one tries to maximize steady-state consumption by choosing saving rate $s$. Which value would one choose?__

__(Bonus) Suppose an agent is given the intertemporal utility from the Ramsey–Cass–Koopmans model see exercise 2) but chooses saving rate once for all. Which saving rate would one choose to maximize intertemporal utility? Is it the same for all initial levels of capital?__



## Deterministic Neoclassical Model: bruteforce

In the Ramsey–Cass–Koopmans  model (also called neoclassical growth model), a representative agent chooses $c_t$ in every period so as to maximize an intertemporal utility like $\sum_{t \geq 0} \beta^t U(c_t)$ where $\beta \in [0,1[$ is a time discount and $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ is the instantaneous felicity.

The setup is exactly the same as in the the Solow model form session 3, but now we look for a time-dependent saving rule, that is a different $s_t \in [0,1[$ for each date. We will try to choose $s_t$ to maximize

$$\sum_{t = 0}^T \beta^t U(c_t) + \sum_{t = T+1}^{\infty} \beta^t U(c_T) $$


We need to choose vector $\mathbf{s} = (s_0, s_1, ... s_T)$ to maximize intertemporal utility. We can assume $\forall t>T s_t= s_T$


In this exercise, we want to define a function `objective(s::AbstractVector)` and maximize objective numerically using a numerical optimization algorithm.


__Explain why the second term is important__

__Choose an horizon $T$ and write a function `objective(s::AbstractVector, k0::Float64, T::Int, p::Dict{Symbol, Float64})` to compute the objective approximated with $T$ periods as a function of initial state `k0::Float64` and of the vector of saving rates `s::Vector` representing $(s_0, s_1, \cdots s_{T})$. Note that argument type of `s` is left unspecified to make it compatible with automatic differentiation.__

__Use library `Optim.jl` to maximize the utility function. If possible, find a way to specify exact derivatives of the objective function.__